In [14]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI, MCSAPI

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("SWAN_API"))

api_key = os.getenv("MCS_API")
mcs_api = MCSAPI(api_key)


In [22]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['Ivano-Frankivsk Oblast-UA',
   'North Rhine-Westphalia-DE',
   'Jeollanam-do-KR',
   'Kowloon City-HK',
   'Kyiv City-UA',
   'North Carolina-US',
   'Beijing-CN',
   'Quebec-CA',
   'California-US',
   'Guangdong-CN',
   'Bavaria-DE',
   'Saxony-DE',
   'North Holland-NL',
   'Jiangsu-CN',
   'Illinois-US',
   'Tokyo-JP'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['Ivano-Frankivsk Oblast-UA',
   'North Rhine-Westphalia-DE',
   'Kowloon City-HK',
   'Kyiv City-UA',
   'North Carolina-US',
   'Beijing-CN',
   'California-US',
   'Guangdong-CN',
   'North Holland-NL',
   'Jiangsu-CN',
   'Illinois-US',
   'Tokyo-JP'],
  'price': '1.0',
  'status': 'available'},
 {'id': 6,
  'name': 'M1ae.2xlarge',
  'description': 'Nvidia 2080 Ti · 4 vCPU · 8 GiB',
  'type': 'GPU'

In [23]:
device = "G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

13
('G1ae.medium', 13, ['Kowloon City-HK', 'Jiangsu-CN', 'Tokyo-JP', 'California-US'])


In [17]:
from swan.object.source_uri import LagrangeSpace

lag = LagrangeSpace('0x5339595102d922AA0F8EfD6DCA9a2b406F57937a', 'NSFWDetection', os.getenv("WALLET"), obj)
 
lag.get_space_info()

# Upload Directory to MCS
response = lag.generate_source_uri('swan_test', 'mar8s1', 'nsfw.json', mcs_client = mcs_api)

In [18]:
print(lag.source_uri)

https://plutotest.acl.swanipfs.com/ipfs/QmPA19sHXnBPHWk2oohQZtogV4zDc8GZPvsv97xQP3eJSj


In [19]:
import sys
sys.path.insert(0, '..')
 
import os
from dotenv import load_dotenv
load_dotenv()
 
from swan import SwanContract
 
pk = os.getenv('PK')
rpc = os.getenv('RPC')
 
c = SwanContract(pk, rpc)
 
# Test esimate lock revenue
r = c.estimate_payment(obj.id, 2)
print(r)

22000000000000000000


In [20]:
r = c._approve_swan_token(22000000000000000000)
print(r)
 
r = c.lock_revenue('1', obj.id, 1)
print(r)

0x5ecbbf3b08f29032d8b13cde63341fd3ac9f865748beb183b4a4d03d8479a705
0x15e6f29619ef18ac7c33d7bcf1c90a997fdb86452416865a3cc06c2dfa8cf20c


In [26]:
# Retrieve List of Hardwares

# for hardware in hardwares:
#     if hardware.name == 'C1ae.medium':
#         print(hardware.region, hardware.price)
        
# Deploy task
result = swan_api.deploy_task(
                              cfg_name=device, 
                              region='California-US', 
                              start_in=60*40, duration=3600*1, 
                              job_source_uri="https://plutotest.acl.swanipfs.com/ipfs/QmPA19sHXnBPHWk2oohQZtogV4zDc8GZPvsv97xQP3eJSj", 
                              paid=11,
                              tx_hash="0x15e6f29619ef18ac7c33d7bcf1c90a997fdb86452416865a3cc06c2dfa8cf20c",
                              wallet_address=os.getenv('WALLET')
                            )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

{'data': {'task': {'created_at': '1711728208', 'end_at': '1711734203', 'leading_job_id': None, 'status': 'created', 'task_detail_cid': 'https://data.mcs.lagrangedao.org/ipfs/QmetWRj8Lu3Z6Ttbc8RvKmuKahGsTfKuvT58Ldd3qEpq5D', 'updated_at': '1711728208', 'uuid': '26f645f7-0cd9-4441-b770-9a168048e832'}}, 'message': 'Task_uuid created.', 'status': 'success'}
Task UUID: 26f645f7-0cd9-4441-b770-9a168048e832


In [27]:
import json

# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_url = info['data']['jobs'][0]['job_result_uri']
        job_real_url = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_url)
        print("Job Real URL: ", job_real_url)
        
        # break
        if status == 'deploytok8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(20)

Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io
Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io
Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io
Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io
Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io
Submitted
Job Result URL:  https://bd8fa49f2935.acl.swanipfs.com/ipfs/Qmd3rDHyNtzW6fEuF3PVYkKfswuG7aRzzkGFjZ9eRsyMaD
Job Real URL:  https://82n07mpltv.lag.topipfs.io


KeyboardInterrupt: 

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

In [15]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'link': [
        'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s',
        'https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png',
        'www.google.com',
        'www.pornhub.com',
    ],
}

response = requests.post(f"http://127.0.0.1:5000/link", headers=headers, json=json_data)
try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


{
    "result": [
        {
            "is_nsfw_image": true,
            "is_nsfw_link": false,
            "link": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s",
            "probability": 0.999750554561615
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png",
            "probability": 0.999238133430481
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "www.google.com",
            "probability": null
        },
        {
            "is_nsfw_image": null,
            "is_nsfw_link": true,
            "link": "www.pornhub.com",
            "probability": 100
        }
    ]
}


In [ ]:
log_path = '../nsfw.log'

if os.path.isfile(log_path):
    with open(log_path, 'w') as f:
        f.write('link,IS NSFW IMAGE,IS NSFW LINK,Probability\n')

with open(log_path, 'a') as f:
    for result in response.json()['result']:
        f.write(f"{result['link']},{result['is_nsfw_image']},{result['is_nsfw_link']},{result['probability']}\n")
        